In [1]:
#Init enviroment
!python3 -m venv OXTC
!source OXTC/bin/activate
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Generate the onnx model
%run generate_onnx_model.py

Creating model with in_size=5 and out_size=3
Model successfully created.
Model saved to ./simple_model_5_3.onnx

Running the model to get test data
Loading model from ./simple_model_5_3.onnx
Input x: shape=[1, 5]
Output y: shape=[1, 3]
Output y_hat: shape=[1, 3]

Weights:
[[  0 -10   0]
 [  7  -3 -17]
 [ -7  -9  -7]
 [  9  10  14]
 [  6  20  18]]

Input:
x:	 [[0 0 1 1 0]]

Outputs:
y:	 [[2 1 7]]
y_hat:	 [[1 1 1]]


In [5]:
#Import depenecys
import json
import tarfile
import pathlib
import tempfile
import numpy as np

import onnx
import tvm
from tvm import te
import tvm.relay as relay
import tvm.micro
import tvm.micro.testing
import tvm.contrib.utils
from tvm.micro import export_model_library_format

#Import model
onnx_model = onnx.load("simple_model_5_3.onnx")

In [6]:
RUNTIME = tvm.relay.backend.Runtime("crt", {"system-lib": True})
TARGET = tvm.micro.testing.get_target("crt")

input_name = "x"
shape_dict = {"x": (5,)}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

with tvm.transform.PassContext(opt_level=3, config={"tir.disable_vectorize": True}):
    module = relay.build(mod, target=TARGET, runtime=RUNTIME, params=params)

Matmul is not optimized for x86. Recommend to use cblas/mkl/dnnl for better performance.


In [8]:
c_source_module = module.get_lib().imported_modules[0]
assert c_source_module.type_key == "c", "tutorial is broken"

c_source_code = c_source_module.get_source()
first_few_lines = c_source_code.split("\n")[:10]
assert any(
    l.startswith("TVM_DLL int32_t tvmgen_default_") for l in first_few_lines
), f"tutorial is broken: {first_few_lines!r}"
print("\n".join(first_few_lines))

temp_dir = tvm.contrib.utils.tempdir()
model_tar_path = temp_dir / "model.tar"
export_model_library_format(module, model_tar_path)

with tarfile.open(model_tar_path, "r:*") as tar_f:
    print("\n".join(f" - {m.name}" for m in tar_f.getmembers()))

template_project_path = pathlib.Path(tvm.micro.get_microtvm_template_projects("crt"))
project_options = {}  # You can use options to provide platform-specific options through TVM.

# Create a temporary directory
temp_dir = tvm.contrib.utils.tempdir()
generated_project_dir = temp_dir / "generated-project"
generated_project = tvm.micro.generate_project(
    template_project_path, module, generated_project_dir, project_options
)

# Build and flash the project
generated_project.build()

// tvm target: c -keys=cpu -model=host
#define TVM_EXPORTS
#include "tvm/runtime/c_runtime_api.h"
#include "tvm/runtime/c_backend_api.h"
#include <math.h>
#include <stdbool.h>
#ifdef __cplusplus
extern "C"
#endif
TVM_DLL int32_t tvmgen_default_fused_expand_dims(void* args, int32_t* arg_type_ids, int32_t num_args, void* out_ret_value, int32_t* out_ret_tcode, void* resource_handle);


NameError: name 'tarfile' is not defined